In [1]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
from netCDF4 import Dataset

import numpy as np
import Nio,Ngl
import os,sys
import datetime


In [2]:
  path_to_output = "/scratch/rhe/GoM/Data-Assimilation/jan2009b/"
  #-- open file
  f = Nio.open_file(path_to_output+"instance_0001/roms_posterior_0001_54939.nc","r")
  gridf = Nio.open_file(path_to_output+"instance_0001/useast_ini_0001.nc","r")
  lats=gridf.variables['lat_rho'][:,0]
  lons=gridf.variables['lon_rho'][0,:]
  wks_type = "png"
  wks = Ngl.open_wks(wks_type,"ssh17_every_14_days")
  colorstouse = ['white','black','grey','blue','cyan3','darkgreen','yellow','brown4','salmon','red','magenta']
  #Ngl.define_colormap(wks,"default")         # Change color map.
    
  Ngl.define_colormap(wks,colorstouse)
  start_date=datetime.datetime(2009,5,24)
  end_date=datetime.datetime(2009,10,8) # Fig 4.5-9
 

  #############################################################
  days_since=datetime.datetime(1858,11,17)
  start_date_ordinal=datetime.date.toordinal(start_date)
  end_date_ordinal=datetime.date.toordinal(end_date)
  days_since_ordinal=datetime.date.toordinal(days_since)
    
  start_date_index=start_date_ordinal-days_since_ordinal
  end_date_index=end_date_ordinal-days_since_ordinal
    
  # Optional sanity check
  #start_date_string=datetime.date.fromordinal(days_since_ordinal+start_date_index)
  nfiles=int((end_date_index-start_date_index+1)/2)

  filesi=list(range(start_date_index,end_date_index,2))
    
  latbound_s  = 23 #21
  latbound_n  = 28 #29
  lonbound_w  = -90 #-91
  lonbound_e  = -84 #-82


  # latitude lower and upper index
  lati_s = (np.abs(lats - latbound_s)).argmin()
  lati_n = (np.abs(lats - latbound_n)).argmin()
  loni_w = (np.abs(lons - lonbound_w)).argmin()
  loni_e = (np.abs(lons - lonbound_e)).argmin()
  lats_sub = lats[lati_s:lati_n]
  lons_sub = lons[loni_w:loni_e]


  h=gridf.variables['h'][lati_s:lati_n,loni_w:loni_e]

    
  mres                     = Ngl.Resources()
  mres.mpLimitMode           = "LatLon"   # Specify area of map
  mres.mpMaxLatF             = latbound_n       # to zoom in on.
  mres.mpMinLatF             = latbound_s
  mres.mpMaxLonF             = lonbound_e        # to zoom in on.
  mres.mpMinLonF             = lonbound_w
  mres.sfXArray             = lons_sub         # Area of map to overlay 
  mres.sfYArray             = lats_sub        # contours on.
  mres.mpProjection        = "Mercator"    # choose projection
  mres.tfDoNDCOverlay      = True                  # native grid, no transform
  mres.nglDraw             = False                 # don't draw yet
  mres.nglFrame            = False                 # don't advance frame yet
  mres.pmTickMarkDisplayMode = "Always"            # turn on tickmarks    

  mres.nglMaximize = False
  mres.vpYF = 0.9


  res = Ngl.Resources()
  for t in dir(mres):
   if (t[0:2] == 'cn' or t[0:2] == 'sf' or t[0:3] == 'ngl'):
    setattr(res,t,getattr(mres,t))

  res.mpLimitMode           = "LatLon"   # Specify area of map

  res.sfXArray             = lons_sub         # Area of map to overlay 
  res.sfYArray             = lats_sub         # contours on.
  plot_base = Ngl.map(wks,mres)

    

      
  res.cnLineColor = 'forestgreen'    
  res.cnLevelSelectionMode             = "ManualLevels"         
  res.cnMinLevelValF       = 0          # contour levels.
  res.cnMaxLevelValF       = 5000.
  res.cnLevelSpacingF      = 500.
  res.cnLineThicknessF     = 1                # thickness of contour lines
  res.cnLineDashPattern    = 2
  res.cnLineLabelsOn       = True            # no line labels
  res.cnLineLabelFontHeightF = 0.007
  res.cnLineLabelInterval = 1
  res.cnInfoLabelOn = False

  plotb = Ngl.contour(wks,h,res)  
  Ngl.overlay(plot_base,plotb)                  # Overlay this contour on map.

  res.cnLevelSelectionMode = "ExplicitLevels" # explicit contour levels
  res.cnLevels             = 0.17             # which level(s) to plot
  res.cnInfoLabelOn        = False            # no info label
  res.cnLineLabelsOn       = False            # no line labels
  res.cnLineThicknessF     = 4                # thickness of contour lines
  res.cnLineColor = 'red'  
  res.cnLineDashPattern    = 0

  colorslist=list(colorstouse)
  dimsc=len(colorslist)
  lotta_labels=[str(item) for item in colorslist[0:dimsc-1]]

  rlist                  = Ngl.Resources()
  rlist.lgAutoManage = True
  rlist.lgBoxMinorExtentF =0.15 # change the size of the lines
  #rlist.lgItemTypes = "Lines"
  #rlist.lgLabelAlignment   = "AboveItems"
  rlist.lgLabelFontHeightF = 0.43
  rlist.lgOrientation     = "Horizontal"
  rlist.lgMonoDashIndex    = True
  rlist.lgLineThicknessF   = 4.0
  rlist.lgBottomMarginF = 0.2
  #rlist.lgBoxMajorExtentF = 0.3
  rlist.lgLabelFontHeightF = .91
  #rlist.LineLengthPercent = 0.01
    
  # Try to get as close to every 15 days as possible
  m = np.empty(10)
  startt = filesi[0]
  print(m)
  for t in range(0,10):
    m[t] = startt
    if t % 2 == 0:
      # Even
      startt = startt+16
    else:
      # Odd
      startt = startt+14
  print(m)    

  for t in range(0,10):
    date_obj=datetime.date.fromordinal(int(m[t])+days_since_ordinal) 
    date_str=date_obj.strftime('%-m/%e')
    lotta_labels[int(t)]=date_str 
    #date_obj=datetime.date.fromordinal(int(filesi[t])+days_since_ordinal)        
    #lotta_labels[int(t/7)]=date_str  

  print('labels',lotta_labels)
    
  #for t in range(0,nfiles,7):
  for t in range(0,10):
    filesi[t]=int(m[t])
    f1t=filesi[t]-4
    f2t=filesi[t]-2
    f4t=filesi[t]+2
    f5t=filesi[t]+4
    print(path_to_output+"analysis_mean."+str(f1t)+".nc")

    f1 = Nio.open_file(path_to_output+"analysis_mean."+str(f1t)+".nc","r")
    f2 = Nio.open_file(path_to_output+"analysis_mean."+str(f2t)+".nc","r")
    f3 = Nio.open_file(path_to_output+"analysis_mean."+str(filesi[t])+".nc","r")
    f4 = Nio.open_file(path_to_output+"analysis_mean."+str(f4t)+".nc","r")
    f5 = Nio.open_file(path_to_output+"analysis_mean."+str(f5t)+".nc","r")

    z = (f1.variables['zeta'][0,lati_s:lati_n,loni_w:loni_e] + 
         f2.variables['zeta'][0,lati_s:lati_n,loni_w:loni_e] +
         f3.variables['zeta'][0,lati_s:lati_n,loni_w:loni_e] +
         f4.variables['zeta'][0,lati_s:lati_n,loni_w:loni_e] +
         f5.variables['zeta'][0,lati_s:lati_n,loni_w:loni_e])/5
    
    #colorstouseindex=int(t/7)+1 #skip white
    colorstouseindex=int(t)+1 # skip white
    #Ngl.draw_ndc_grid(wks)
    res.cnLineColor = colorstouse[colorstouseindex]                      # Change line color.
    print(colorstouse[colorstouseindex])
    lg = Ngl.legend_ndc(wks,len(lotta_labels),lotta_labels,0.2,0.4,rlist)  

    plot = Ngl.contour(wks,z,res)           # Generate contours.
    Ngl.overlay(plot_base,plot)                  # Overlay this contour on map. 
    Ngl.draw(plot_base)
    Ngl.frame(wks) 
    
  #Ngl.draw(plot_base)
  #Ngl.frame(wks)  
    
  #from IPython.display import Image
  #Image("ssh_every_14_days.png")

[2.33761252e-310 2.33761252e-310 4.65232477e-310 0.00000000e+000
 4.65232477e-310 4.65232477e-310 3.39519327e-313 1.63041663e-322
 4.65232477e-310 4.65232476e-310]
[54975. 54991. 55005. 55021. 55035. 55051. 55065. 55081. 55095. 55111.]
labels ['5/24', '6/ 9', '6/23', '7/ 9', '7/23', '8/ 8', '8/22', '9/ 7', '9/21', '10/ 7']
/scratch/rhe/GoM/Data-Assimilation/jan2009b/analysis_mean.54971.nc
black
/scratch/rhe/GoM/Data-Assimilation/jan2009b/analysis_mean.54987.nc
grey
/scratch/rhe/GoM/Data-Assimilation/jan2009b/analysis_mean.55001.nc
blue
/scratch/rhe/GoM/Data-Assimilation/jan2009b/analysis_mean.55017.nc
cyan3
/scratch/rhe/GoM/Data-Assimilation/jan2009b/analysis_mean.55031.nc
darkgreen
/scratch/rhe/GoM/Data-Assimilation/jan2009b/analysis_mean.55047.nc
yellow
/scratch/rhe/GoM/Data-Assimilation/jan2009b/analysis_mean.55061.nc
brown4
/scratch/rhe/GoM/Data-Assimilation/jan2009b/analysis_mean.55077.nc
salmon
/scratch/rhe/GoM/Data-Assimilation/jan2009b/analysis_mean.55091.nc
red
/scratch/rhe/Go